In [4]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import AIPluginTool

from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access the environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# Access the environment variables
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')


In [3]:
# Setup environment variables
URL = "https://blockchatstatic.blob.core.windows.net/api-configuration"

llm = ChatOpenAI(temperature=0)

In [15]:
tools = load_tools(["requests_post"], allow_dangerous_tools=True)

# AIPluginTool only fetches and returns the openapi.yaml linked to in /.well-known/ai-plugin.json
# This may need some more work to avoid blowing up LLM context window
solana_search_tool = AIPluginTool.from_plugin_url(URL + "/.well-known/ai-plugin.json")
tools += [solana_search_tool]

web_search_tool = TavilySearchResults(k=3)
tools += [web_search_tool]

llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

In [ ]:
# AI Agent does best when it only has one available tool
# to engage with URLs
tools = load_tools(["requests_post"], allow_dangerous_tools=True)

# AIPluginTool only fetches and returns the openapi.yaml linked to in /.well-known/ai-plugin.json
# This may need some more work to avoid blowing up LLM context window
tool = AIPluginTool.from_plugin_url(URL + "/.well-known/ai-plugin.json")
tools += [tool]

In [6]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [13]:
# Setup an agent to answer the question without further human feedback
agent = create_react_agent(
    llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True)

agent_executor.invoke({"input": "How many lamports does 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 have?"})

{'input': 'How many lamports does 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 have?',
 'output': 'The address 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 has a balance of 0.042545265 SOL.'}

In [10]:
agent_executor.invoke({"input": "What is Kamino Finance??"})

{'input': 'What is Kamino Finance??',
 'output': 'Kamino Finance is a decentralized finance protocol on Solana that provides various DeFi products such as borrowing, lending, and leveraging assets.'}

In [11]:
agent_executor.invoke({"input": "Hello"})

{'input': 'Hello',
 'output': 'Agent stopped due to iteration limit or time limit.'}